In [1]:
map = {
'30824':'Indicadores de renta media',
'30825':'Distribución por fuente de ingresos',
'30826':'Distribución de la renta por unidad de consumo',
'30827':['Distribución de la renta por unidad de consumo','Sexo','Tramos de edad'],
'30828':['Distribución de la renta por unidad de consumo','Sexo','Nacionalidad'],
'30829':['Distribución de la renta por unidad de consumo','Sexo'],
'30830':['Distribución de la renta por unidad de consumo','Sexo','Tramos de edad'],
'30831':['Distribución de la renta por unidad de consumo','Sexo','Nacionalidad'],
'30832':'Indicadores demográficos',
'37677':'Indicadores de renta media'}

variables = ['Renta neta media por persona ', 'Renta neta media por hogar',
'Media de la renta por unidad de consumo',
'Mediana de la renta por unidad de consumo',
'Renta bruta media por persona', 'Renta bruta media por hogar',
'Fuente de ingreso: salario', 'Fuente de ingreso: pensiones',
'Fuente de ingreso: prestaciones por desempleo',
'Fuente de ingreso: otras prestaciones',
'Fuente de ingreso: otros ingresos',
'Edad media de la población',
'Porcentaje de población menor de 18 años',
'Porcentaje de población de 65 y más años',
'Tamaño medio del hogar',
'Porcentaje de hogares unipersonales',
'Población',
'Porcentaje de población española',
'Índice de Gini',
'Distribución de la renta P80/P20']

In [2]:
path = r"C:\Users\ManuBenito\Documents\Walknet-DataLake\sources\ine_atlasrenta"

In [3]:
import pandas as pd

In [4]:
def download_atlasrenta(path):
    for key, values in map.items():
        url = "https://www.ine.es/jaxiT3/files/t/es/csv_bdsc/{key}.csv?nocab=1"
        pd.read_csv(url,sep=";").to_csv(f"{path}\level0\{key}_{values[0]}.csv",sep=";")

In [5]:
def process_ineatlas_data(path):
    dfs = []
    for key, fields in map.items():
        df = pd.read_csv(f"{path}\level0\{key}.csv", sep =";")
        df = df[~df['Secciones'].isna()]
        df['code'] = df['Secciones'].str.split(" ").str[0]
        df['name'] = df['Secciones'].str.split(" ", n=1).str[1]
        df['period'] = df['Periodo']
        if isinstance(fields,list):
            df['variable'] = df[fields].agg(' - '.join, axis=1)
        else:
            df['variable'] = df[fields]
        df['value'] = df['Total']
        dfs.append(df[['code','name','period','variable','value']])
    pd.concat(dfs).to_parquet(path=f"{path}\level1\ine_atlasrenta.parquet")

In [19]:
def transform_ineatlas_data(path):
    
    df = pd.read_parquet(f"{path}\level1\ine_atlasrenta.parquet")

    weird_values = list(df[~df['value'].fillna('-999').str.contains("[0-9]\.[0-9]|[0-9]\,[0-9]|^[0-9]+$",regex=True)]['value'].unique())

    df.loc[df['value'].isin(weird_values),'value'] = '-999'

    df['value'] = df['value'].fillna('-999').str.replace(".","").str.replace(",",".").astype(float)
    
    df[df['variable'].isin(variables)]
    
    return df